In [2]:
import csv
import pandas as pd
import torch
import torch.nn.functional as F

# load the names from dataset/indianames.csv
names = []
with open('dataset/indianames.csv', 'r') as f:
    names = f.readlines()
names = [name.strip() for name in names]
names.sort()

xs = []
ys = []
stoi = {c: i for i, c in enumerate('.abcdefghijklmnopqrstuvwxyz')}
itos = {i: c for c, i in stoi.items()}
block_size = 3

for name in names:
    chars = list(f"...{name}.")
    for i in range(len(chars) - block_size):
        
        xs.append([stoi[c] for c in chars[i:i+block_size]])
        ys.append(stoi[chars[i+block_size]])
    




In [73]:
X = torch.tensor(xs)
Y = torch.tensor(ys)

# lookup table C for embedding
emb_dim = 2
C = torch.randn(27, emb_dim, dtype=torch.float)

W1 = torch.randn((emb_dim * block_size,300))
b1 = torch.randn(300)

W2 = torch.randn((300, 27))
b2 = torch.randn(27)


parameters = [C, W1, b1, W2, b2]

x_enc = F.one_hot(X, num_classes=len(stoi)).float()

# enable gradient calculation
for p in parameters:
    p.requires_grad = True

In [77]:
lr = 0.003
for _ in range(100000):

    # mini batch
    ix = torch.randint(0, X.shape[0], (256,))

    # forward pass
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, block_size*2) @ W1 + b1)
    logits = h @ W2 + b2
    # negative log likelihood loss - self implementation
    # counts = logits.exp()
    # probs = counts / counts.sum(dim=1, keepdim=True)
    # loss = -probs[torch.arange(48775), Y].log().mean()
    loss = F.cross_entropy(logits, Y[ix])
    # cross entropy function makes forward and backward pass more efficient

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update weights
    for p in parameters:
        p.data += -lr * p.grad
    
print(loss.item())


1.8243987560272217


In [78]:

for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1)
        context = context[1:] + [ix.item()]
        out.append(ix.item())
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

krip.
kak.
guran.
preen.
chrilu.
yush.
nijami.
chun.
ladabubeda.
masmal.
parsda.
vilna.
istaruj.
karsh.
gamu.
dhal.
josibul.
jiy.
dur.
chote.
